In [8]:
import random
from greedy import greedy_cit

In [103]:
class TestSuite():
    def __init__(self, base, additional):
        self.base = base
        self.additional = additional
        self.fitness_value = None
        
    def assign_fitness_value(self, fitness_value):
        self.fitness_value = fitnes_value
        
    def make_test_suite(self):
        self.test_suite = self.base + self.additional
        
    def eval_variety(self, parameter_num, parameter_range):
        appear_time = []
        
        for ith_param in range(parameter_num):
            value_list = [1 for _ in range(parameter_range+1)]
            appear_time.append(value_list)
            
        for testcase in self.additional:
            for ind in range(len(testcase)):
                appear_time[ind][testcase[ind]] += 1
                
        score = 1
        for ith_param in appear_time:
            for ind in range(len(ith_param)):
                if ith_param[ind] != 1:
                    score = score * ((ith_param[ind]*parameter_num)/len(self.additional))
                    #prevent the fitness value become too big or too small for temprorary
                    #if there are better standard, need to modify
                
        self.fitness_value = score
    
    #assume soft_list format [constraint1 = [[parameter index,lower range, upper range], [], ...], constraint2 = []...]
    #if test case satisfy constraint condition, multiply fitness value 1.3
    def eval_soft_constraint(self, soft_list):
        for constraint in soft_list:
            for testcase in self.additional:
                for (ind, condition) in enumerate(constraint):
                    if testcase[condition[0]] < condition[1] or condition[2] < testcase[condition[0]]:
                        break
                    if ind == (len(constraint)-1):
                        self.fitness_value = self.fitness_value * 1.3
                    
        
    #hard_list format is same as soft_list
    #if test case satisfy hard constraint condition, set fitness value to 0 and return
    def eval_hard_constraint(self, hard_list):
        for constraint in hard_list:
            for testcase in self.additional:
                for (ind, condition) in enumerate(constraint):
                    if testcase[condition[0]] < condition[1] or condition[2] < testcase[condition[0]]:
                        break
                    if ind == (len(constraint)-1):
                        self.fitness_value = 0
                        return
                
    def print_fit_val(self):
        print(self.fitness_value)
                

In [141]:
class GA():
    # Range for budget
    # len(base) < budget == len(base) + len(additional) < len(greedy_suite)
    def __init__(self, greedy_suite, base, parameter_num, parameter_range, budget, population_num, mutation_rate):
        self.greedy_suite = greedy_suite
        self.base = base
        self.parameter_num = parameter_num
        # maximum value of the parameter
        self.parameter_range = parameter_range
        self.budget = budget
        self.mutation_rate = mutation_rate
        self.population_num = population_num
        self.fitness_result =[]
        self.population = []
        
        
    # must do this
    def delete_base_from_greedy(self):
        for i in range(len(self.base)):
            target = self.base[i]
            if(target in self.greedy_suite):
                self.greedy_suite.remove(target)
    # 수정 필요
    def initial_population(self):
        #Using Approach 1: randomly choosing from the greedy - test suite
        initial_population = []
        for i in range(self.population_num):
            gene = random.choices(self.greedy_suite, k = self.budget - len(self.base))
            new_suite = TestSuite(self.base, gene)
            initial_population.append(new_suite)
        self.population = initial_population
            
    def fitness_calculate(self, soft_list, hard_list):
        
        #print("fitness calculating this population")
        #for i in range(len(self.population)):
        #    print(self.population[i].additional)
        self.fitness_result = []
        for i in range(self.population_num):
            self.population[i].eval_variety(self.parameter_num, self.parameter_range)
            self.population[i].eval_soft_constraint(soft_list)
            self.population[i].eval_hard_constraint(hard_list)
            self.fitness_result.append(self.population[i].fitness_value)
    
    def select_parent(self):
        result = random.choices(self.population, weights = self.fitness_result, k = 2)

        return result
    
    
    def make_children(self, parent1, parent2):
        #Sort the list lexicographically
        # For now, just take half / half
        sorted_parent1 = sorted(parent1.additional, key = lambda elem: elem)
        sorted_parent2 = sorted(parent2.additional, key = lambda elem: elem)
        
        child1 = []
        child2 = []


        assert(self.budget - len(self.base) == len(parent1.additional))
        assert(self.budget - len(self.base) == len(parent2.additional))
        additional_length = self.budget - len(self.base)
        for i in range(additional_length):
            if(i < int(additional_length / 2)):
                child1.append(sorted_parent1[i])
                child2.append(sorted_parent2[i])
            else:
                child1.append(sorted_parent2[i])
                child2.append(sorted_parent1[i])
                
                
        return child1
    
    def mutate_test_suite(self, test_suite):
        #iterate for each parameter
        mutant = []
        for i in range(len(test_suite[0])):
            new_dict = {}
            for j in range(len(test_suite)):
                if(test_suite[j][i] not in new_dict.keys()):
                    new_dict[test_suite[j][i]] = 0
                else:
                    new_dict[test_suite[j][i]] = new_dict[test_suite[j][i]] + 1
            minimum_cnt = -1
            minimum_var = None
            for j in range(len(new_dict.keys())):
                temp = list(new_dict.keys())[j]
                temp_cnt = new_dict[temp]
                if(minimum_cnt < 0 or temp_cnt < minimum_cnt):
                    minimum_cnt = temp_cnt
                    minimum_var = temp
            
            mutant.append(minimum_var)
        del test_suite[0]
        test_suite.append(mutant)
        return test_suite
        
    def mutate_population(self, new_population):
        mutated_population = []
        for i in range(len(new_population)):
            target_test_suite = new_population[i].additional
            if(random.random() < self.mutation_rate):
                new_test_suite = self.mutate_test_suite(target_test_suite)
                mutated_population.append(TestSuite(self.base, new_test_suite))
            else:
                mutated_population.append(TestSuite(self.base, target_test_suite))
                
        return mutated_population
                
    def next_generation(self):

        elite_num = int(self.population_num * 4 / 10)

        children_num = int(self.population_num * 4 / 10)

        random_children_num = self.population_num - elite_num - children_num

        
        sorted_population = sorted(self.population, key = lambda test_suite: test_suite.fitness_value)   
        elite_children = sorted_population[self.population_num - elite_num:]
        print(sorted_population[-1].fitness_value)
       # for i in range(len(sorted_population)):
        #    print(sorted_population[i].fitness_value)

        
        children = []
        for i in range(children_num):
            [parent1, parent2] = self.select_parent()
            child = self.make_children(parent1, parent2)
            children.append(TestSuite(self.base, child))


        
        random_children = []
        for i in range(random_children_num):
            random_child = random.choice(self.population)
            random_children.append(random_child)
            

        new_population = elite_children + children + random_children
        
        mutated_population = self.mutate_population(new_population)
        
    #    print("previous generation")
    #    for i in range(len(self.population)):
    #        print(self.population[i].additional)
    #    print("next_generation")
    #    for i in range(len(self.population)):
    #        print(new_population[i].additional)
        
        self.population = mutated_population
        
    def adjust(self):

        for i in range(len(self.population)):
            target_test_suite = self.population[i].additional
            target_set = set(tuple(element) for element in target_test_suite)
            target_lst = [list(t) for t in set(tuple(element) for element in target_set)]
            need_cnt = len(target_test_suite) - len(target_lst)
            for i in range(need_cnt):
                new_case = random.choice(self.greedy_suite)
                while(new_case in target_lst):
                    new_case = random.choice(self.greedy_suite)
                target_lst.append(new_case)
            
            self.population[i].additional = target_lst
            
    
    

In [142]:
a = (greedy_cit(2, ["a1", "a2", "a3", "a4", "a5"], [10, 10, 10, 10, 10]))                 
print(len(a))


146


In [143]:
b = (greedy_cit(3, ["a1", "a2", "a3", "a4", "a5"], [10, 10, 10, 10, 10]))     
print(len(b))                                                  

1598


In [144]:
temp = GA(b, a, 5, 10, 900, 600, 0.5)
print(len(a))
print(len(b))

146
1598


In [145]:
temp.delete_base_from_greedy()

In [146]:
temp.initial_population()

In [147]:


for i in range(100):
    print(i)
    temp.fitness_calculate([],[])
    temp.next_generation()
    temp.adjust()


0
9.232261695078347e-16
1
9.61376604593129e-16
2
1.0397403413810927e-15
3
1.0397403413810927e-15
4
4.567263256083501e-15
5
4.567263256083501e-15
6
4.631991865029646e-15
7
4.631991865029646e-15
8
4.631991865029646e-15
9
4.680184129370398e-15
10
4.680184129370398e-15
11
4.733178769193682e-15
12
4.7865757579734445e-15
13
5.1633282100949455e-15
14
5.569817935000885e-15
15
5.797622712274587e-15
16
6.135294085707211e-15
17
6.382150150857059e-15
18
7.070251206700942e-15
19
7.582407037571705e-15
20
7.988629191879021e-15
21
8.327879451771322e-15
22
8.527120045425686e-15
23
8.543556850572925e-15
24
8.572380370370516e-15
25
8.6326278765493e-15
26
8.6326278765493e-15
27
8.632627876549305e-15
28
8.632627876549305e-15
29
8.632627876549305e-15
30
8.632627876549308e-15
31
8.63382286933399e-15
32
8.633822869333991e-15
33
8.633822869333991e-15
34
8.633822869333991e-15
35
8.633822869333993e-15
36
8.633822869333993e-15
37
8.633822869333993e-15
38
8.633822869333993e-15
39
8.633822869333993e-15
40
8.6338228

In [148]:
temp.fitness_calculate([],[])
sorted_population = sorted(temp.population, key = lambda test_suite: test_suite.fitness_value)   
with open('gentic_cit_900_600_100.txt', 'w') as f:
    f.write(str(sorted_population[-1].base + sorted_population[-1].additional))
print(len(sorted_population[-1].base + sorted_population[-1].additional))

900


Random 

In [149]:
random_population = random.choices(b, k = 900)
with open('random.txt', 'w') as f:
    f.write(str(random_population))
print(len(random_population))

900


Just GA
self, greedy_suite, base, parameter_num, parameter_range, budget, population_num, mutation_rate):

In [150]:
temp2 = GA(b, [], 5, 10, 900, 600, 0.5)
temp2.delete_base_from_greedy()
temp2.initial_population()

In [151]:
for i in range(100):
    print(i)
    temp2.fitness_calculate([],[])
    temp2.next_generation()
    temp2.adjust()


0
8.107572129166864e-16
1
8.170622454858203e-16
2
8.419639220455665e-16
3
9.019670615962934e-16
4
9.083275334678384e-16
5
3.697250785756632e-15
6
9.716616598466034e-16
7
9.886934633789175e-16
8
9.955931150004511e-16
9
1.0048779601255658e-15
10
1.0397589199113148e-15
11
1.0957437772251794e-15
12
1.175244692134871e-15
13
1.2644205113126756e-15
14
1.3854065392326501e-15
15
1.46189037455599e-15
16
1.484965148576329e-15
17
1.5090192112635516e-15
18
1.5196946883158175e-15
19
1.5235749773438585e-15
20
1.5254125892677744e-15
21
1.5254125892677744e-15
22
1.5293629984125765e-15
23
1.5304557098179593e-15
24
1.5311717124892191e-15
25
5.7498379481494395e-15
26
5.82526728129881e-15
27
5.881284819585539e-15
28
5.993129725562817e-15
29
6.080773833558327e-15
30
6.1903300751056984e-15
31
6.301861188398272e-15
32
6.455799582691032e-15
33
6.506419066492705e-15
34
6.60614179263327e-15
35
6.648806971752786e-15
36
6.767914163379768e-15
37
6.877170360505534e-15
38
7.0227898550708964e-15
39
7.104852015594095e-

In [152]:
temp2.fitness_calculate([],[])
sorted_population2 = sorted(temp2.population, key = lambda test_suite: test_suite.fitness_value)   
with open('ga_cit_900_600_100.txt', 'w') as f:
    f.write(str(sorted_population2[-1].additional))
print(len(sorted_population2[-1].additional))

900
